In [1]:
import os
import sys
import numpy as np
import theano.tensor as T
import codecs
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu1")
from collections import Counter
import math
import copy
import sys

Using gpu device 1: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4004)


In [2]:
sys.path.append('/usr0/home/glample/Research/perso/UltraDeep/')

In [3]:
from network import LSTM
from layer import HiddenLayer, EmbeddingLayer
from learning_method import LearningMethod

In [4]:
path_to_oracle = '/usr0/home/glample/Research/IncrementalMT/oracle.tsv'

In [5]:
lines = [line.strip().split(' ||| ') for line in codecs.open(path_to_oracle, 'r', encoding='utf8')]

In [6]:
sentences = []
curr_sentence_pointer = 0
for ind, line in enumerate(lines):
    if len(line) == 1 and line[0] == '':
        sentences.append(lines[curr_sentence_pointer:ind])
        curr_sentence_pointer = ind + 1

In [7]:
train_sentences = sentences[:int(0.95 * len(sentences))]
dev_sentences = sentences[int(0.95 * len(sentences)):int(0.97 * len(sentences))]
test_sentences = sentences[int(0.97 * len(sentences)):]

In [8]:
source_words = set()
for sentence in sentences:
    source_sentence = sentence[0][0]
    for word in source_sentence.split():
        source_words.add(word)
source_words.add('<s>')
source_words.add('</s>')
source_words.add('$NONE$')
source_words.add('<ss>')
source_words.add('</se>')
source_words.add('<fcs>')
source_words.add('</fcs>')

In [9]:
target_words = set()
for sentence in sentences:
    target_sentence = sentence[1][0]
    for word in target_sentence.split():
        target_words.add(word)
target_words.add('<s>')
target_words.add('</s>')
target_words.add('$NONE$')

In [10]:
for sentence in sentences:
    for state in sentence[2:]:
        assert len(state) == 5

In [11]:
source_word2ind = {word:ind for ind, word in enumerate(source_words)}
source_ind2word = {ind:word for ind, word in enumerate(source_words)}
target_word2ind = {word:ind for ind, word in enumerate(target_words)}
target_ind2word = {ind:word for ind, word in enumerate(target_words)}

In [12]:
'''
source_word2ind['<s>'] = len(source_word2ind)
source_word2ind['</s>'] = len(source_word2ind) + 1
source_word2ind['$NONE$'] = len(source_word2ind) + 2
source_word2ind['<ss>'] = len(source_word2ind) + 3
source_word2ind['</se>'] = len(source_word2ind) + 4
source_word2ind['<fcs>'] = len(source_word2ind) + 5
source_word2ind['</fce>'] = len(source_word2ind) + 6

target_word2ind['<s>'] = len(target_word2ind)
target_word2ind['</s>'] = len(target_word2ind) + 1
target_word2ind['$NONE$'] = len(target_word2ind) + 2

source_ind2word[len(source_word2ind)] = '<s>'
source_ind2word[len(source_word2ind) + 1] = '</s>'
source_ind2word[len(source_word2ind) + 2] = '$NONE$'
source_ind2word[len(source_word2ind) + 3] = '<ss>'
source_ind2word[len(source_word2ind) + 4] = '</se>'
source_ind2word[len(source_word2ind) + 5] = '<fcs>'
source_ind2word[len(source_word2ind) + 6] = '</fce>'

target_ind2word[len(target_word2ind)] = '<s>' 
target_ind2word[len(target_word2ind) + 1] = '</s>' 
target_ind2word[len(target_word2ind) + 2] = '$NONE$'
'''

"\nsource_word2ind['<s>'] = len(source_word2ind)\nsource_word2ind['</s>'] = len(source_word2ind) + 1\nsource_word2ind['$NONE$'] = len(source_word2ind) + 2\nsource_word2ind['<ss>'] = len(source_word2ind) + 3\nsource_word2ind['</se>'] = len(source_word2ind) + 4\nsource_word2ind['<fcs>'] = len(source_word2ind) + 5\nsource_word2ind['</fce>'] = len(source_word2ind) + 6\n\ntarget_word2ind['<s>'] = len(target_word2ind)\ntarget_word2ind['</s>'] = len(target_word2ind) + 1\ntarget_word2ind['$NONE$'] = len(target_word2ind) + 2\n\nsource_ind2word[len(source_word2ind)] = '<s>'\nsource_ind2word[len(source_word2ind) + 1] = '</s>'\nsource_ind2word[len(source_word2ind) + 2] = '$NONE$'\nsource_ind2word[len(source_word2ind) + 3] = '<ss>'\nsource_ind2word[len(source_word2ind) + 4] = '</se>'\nsource_ind2word[len(source_word2ind) + 5] = '<fcs>'\nsource_ind2word[len(source_word2ind) + 6] = '</fce>'\n\ntarget_ind2word[len(target_word2ind)] = '<s>' \ntarget_ind2word[len(target_word2ind) + 1] = '</s>' \ntarget_

In [13]:
stack_input = T.ivector()
forward_context_input = T.ivector()
target_input = T.ivector()
gold_action = T.scalar()
target_output = T.ivector()

In [14]:
# Generate synthetic data to test dimensions
syn_stack_input = np.random.randint(low=0, high=len(source_word2ind), size=(5,)).astype(np.int32)
#syn_stack_input = np.array([61699, 49379, 61701]).astype(np.int32)
syn_forward_context_input = np.random.randint(low=0, high=len(source_word2ind), size=(6,)).astype(np.int32)
syn_action_prediction = np.random.randint(low=0, high=2, size=(1,)).astype(np.float32)[0]
syn_target_input = np.random.randint(low=0, high=len(target_word2ind), size=(7,)).astype(np.int32)
syn_target_output = np.random.randint(low=0, high=len(target_word2ind), size=(7,)).astype(np.int32)

In [15]:
# Neural Inventory

# Model Parameters
src_emb_dim      = 256  # source word embedding dimension
tgt_emb_dim      = 256  # target word embedding dimension
src_lstm_hid_dim = 512  # source LSTMs hidden dimension
tgt_lstm_hid_dim = 2 * 512  # target LSTM hidden dimension
beta = 500 # RNN regularization parameter

# Embedding Layers
source_embedding = EmbeddingLayer(input_dim=len(source_word2ind) + 1, output_dim=src_emb_dim)
target_embedding = EmbeddingLayer(input_dim=len(target_word2ind) + 1, output_dim=tgt_emb_dim)

# Stack BiLSTM
stack_source_lstm_forward = LSTM(input_dim=src_emb_dim, hidden_dim=src_lstm_hid_dim, name='source_stack_lstm_forward', with_batch=False)
stack_source_lstm_backward = LSTM(input_dim=src_emb_dim, hidden_dim=src_lstm_hid_dim, name='source_stack_lstm_backward', with_batch=False)

# Forward Context BiLSTM
forward_context_lstm = LSTM(input_dim=src_emb_dim, hidden_dim=src_lstm_hid_dim, name='source_forward_context_lstm', with_batch=False)

# Target Language LSTM Decoder
target_lstm = LSTM(input_dim = tgt_emb_dim, hidden_dim=tgt_lstm_hid_dim, name='target_lstm', with_batch=False)

# Action prediction Layer
action_prediction_weights = HiddenLayer(input_dim=2 * stack_source_lstm_forward.hidden_dim + forward_context_lstm.hidden_dim, output_dim=1)

# Target Language Word Prediction Layer
target_word_decoding_weights = HiddenLayer(input_dim=tgt_lstm_hid_dim + 2 * src_lstm_hid_dim, output_dim=len(target_word2ind), activation='softmax')

# Layer to project the Encoder representation to the same size as h_0 of the decoder LSTM
projection_weights = HiddenLayer(input_dim = 2 * src_lstm_hid_dim, output_dim = tgt_lstm_hid_dim, activation='tanh')

# Set model parameters
action_params = source_embedding.params + stack_source_lstm_forward.params + stack_source_lstm_backward.params + forward_context_lstm.params + action_prediction_weights.params
seq_seq_params = source_embedding.params + stack_source_lstm_forward.params + stack_source_lstm_backward.params + target_embedding.params + target_lstm.params[:-1] + projection_weights.params + target_word_decoding_weights.params

In [16]:
# =====================================================
# The computational graph for this method
# =====================================================

# Get the embedding matrices seq_len x embdding_dim
stack_embedding_matrix = source_embedding.link(stack_input)
print 'stack_embedding_matrix', stack_embedding_matrix.eval({stack_input:syn_stack_input}).shape
forward_context_embedding_matrix = source_embedding.link(forward_context_input)

# ===========================================================
# Get LSTM representations of the stack, forward context
# ===========================================================

# Stack Representation
stack_source_lstm_forward.link(stack_embedding_matrix)
stack_source_lstm_backward.link(stack_embedding_matrix[::-1, :])
stack_lstm_representation = T.concatenate((stack_source_lstm_forward.h, stack_source_lstm_backward.h[::-1,:]), axis=1)
print 'stack_lstm_representation', stack_lstm_representation.eval({stack_input:syn_stack_input}).shape

# Forward Context Representation
forward_context_lstm.link(forward_context_embedding_matrix[::-1, :])
#forward_context_representation = T.concatenate((forward_context_lstm_forward.h, forward_context_lstm_backward.h[::-1,:]), axis=1)
print 'forward_context_lstm_representation', forward_context_lstm.h.eval({forward_context_input:syn_forward_context_input}).shape

# Concatenate representations and make a prediction about what action to take
concatenated_representation = T.concatenate((stack_lstm_representation[-1], forward_context_lstm.h[-1]))
print 'concatenated_representation', concatenated_representation.eval({stack_input:syn_stack_input, forward_context_input:syn_forward_context_input}).shape
action_prediction = action_prediction_weights.link(concatenated_representation).mean() # .mean() is a hack to make it a scalar (since its only a single value, it shouldn't matter)
print 'prediction', action_prediction.eval({stack_input:syn_stack_input, forward_context_input:syn_forward_context_input})

# Compute squared-error loss between predicted action and gold action
action_prediction_loss = ((gold_action - action_prediction) ** 2).mean()
print 'action_prediction_loss', action_prediction_loss.eval({stack_input:syn_stack_input, forward_context_input:syn_forward_context_input, gold_action:syn_action_prediction})

# Get target input embeddings
target_embeddings = target_embedding.link(target_input)
print 'target_embeddings', target_embeddings.eval({target_input:syn_target_input}).shape

target_lstm.h_0 = projection_weights.link(stack_lstm_representation[-1])
print 'target_lstm_h0', target_lstm.h_0.eval({stack_input:syn_stack_input}).shape
#repeated_src_context = T.repeat(stack_lstm_representation[-1].dimshuffle('x', 0), target_embeddings.shape[0], axis=0)
#print 'repeated_src_context', repeated_src_context.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape
#repeated_src_context = projection_weights.fprop(repeated_src_context)
#print 'repeated_src_context', repeated_src_context.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape
#target_embeddings = T.concatenate((target_embeddings, repeated_src_context), axis=1)
#print 'target_embeddings', target_embeddings.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape
target_lstm.link(target_embeddings)
print 'target_lstm', target_lstm.h.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

transition = target_lstm.h.dot(stack_lstm_representation.transpose())
transition = transition.dot(stack_lstm_representation)
print 'transition', transition.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

transition_last = T.concatenate([transition, target_lstm.h], axis=1)
decoded_words = target_word_decoding_weights.link(transition_last)
print 'decoded_words', decoded_words.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape
decoding_loss = T.nnet.categorical_crossentropy(decoded_words, target_output).mean()
decoding_loss += beta * T.mean((target_lstm.h[:-1] ** 2 - target_lstm.h[1:] ** 2) ** 2) # Regularization of RNNs from http://arxiv.org/pdf/1511.08400v6.pdf

print 'decoding_loss', decoding_loss.eval({stack_input:syn_stack_input, target_input:syn_target_input, target_output:syn_target_output})

'''
# Appened source representation to input at every step
repeated_src_context = T.repeat(stack_lstm_representation[-1].dimshuffle('x', 0), target_embeddings.shape[0], axis=0)
#repeated_src_context = proj_layer2.link(repeated_src_context)
target_embeddings = T.concatenate((target_embeddings, repeated_src_context), axis=1)

# Get the decoded sentence by connecting Encoder & Decoder
connection = projection_weights.link(stack_lstm_representation[-1])
print 'connection', connection.eval({stack_input:syn_stack_input}).shape
target_lstm.h_0 = connection
target_lstm.link(target_embeddings)
print 'target_lstm', target_lstm.h.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

transition = target_lstm.h.dot(stack_lstm_representation.transpose())
transition = transition.dot(stack_lstm_representation)
print 'transition', transition.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

# Decode words
decoded_words = target_word_decoding_weights.link(target_lstm.h)
print 'decoded_words', decoded_words.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape

# Compute seq-seq loss
decoding_loss = T.nnet.categorical_crossentropy(decoded_words, target_output).mean()
'''

stack_embedding_matrix (5, 256)
stack_lstm_representation (5, 1024)
forward_context_lstm_representation (6, 512)
concatenated_representation (1536,)
prediction 0.499724894762
action_prediction_loss 0.249724969268
target_embeddings (7, 256)
target_lstm_h0 (1024,)
target_lstm (7, 1024)
transition (7, 1024)
decoded_words (7, 71180)
decoding_loss 11.1729450226


"\n# Appened source representation to input at every step\nrepeated_src_context = T.repeat(stack_lstm_representation[-1].dimshuffle('x', 0), target_embeddings.shape[0], axis=0)\n#repeated_src_context = proj_layer2.link(repeated_src_context)\ntarget_embeddings = T.concatenate((target_embeddings, repeated_src_context), axis=1)\n\n# Get the decoded sentence by connecting Encoder & Decoder\nconnection = projection_weights.link(stack_lstm_representation[-1])\nprint 'connection', connection.eval({stack_input:syn_stack_input}).shape\ntarget_lstm.h_0 = connection\ntarget_lstm.link(target_embeddings)\nprint 'target_lstm', target_lstm.h.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape\n\ntransition = target_lstm.h.dot(stack_lstm_representation.transpose())\ntransition = transition.dot(stack_lstm_representation)\nprint 'transition', transition.eval({stack_input:syn_stack_input, target_input:syn_target_input}).shape\n\n# Decode words\ndecoded_words = target_word_decoding_we

In [17]:
'''
Update parameters using ADAM
'''
updates_action =LearningMethod(clip=5.0).get_updates('adam', action_prediction_loss, action_params)

In [18]:
'''
Update parameters using ADAM
'''
updates_seq_seq = LearningMethod(clip=5.0).get_updates('adam', decoding_loss, seq_seq_params)

In [19]:
f_train_action = theano.function(
    inputs=[stack_input, forward_context_input, gold_action],
    outputs=[action_prediction_loss, action_prediction],
    updates=updates_action
)

In [20]:
f_train_seq_seq = theano.function(
    inputs=[stack_input, target_input, target_output],
    outputs=decoding_loss,
    updates=updates_seq_seq
)

In [21]:
f_get_action = theano.function(
    inputs=[stack_input, forward_context_input],
    outputs=action_prediction,
)

In [22]:
f_eval_seq_seq = theano.function(
    inputs=[stack_input, target_input],
    outputs=decoded_words,
)

In [23]:
def bleu_stats(hypothesis, reference):
    stats = []
    stats.append(len(hypothesis))
    stats.append(len(reference))
    for n in xrange(1,5):
        s_ngrams = Counter([tuple(hypothesis[i:i+n]) for i in xrange(len(hypothesis)+1-n)])
        r_ngrams = Counter([tuple(reference[i:i+n]) for i in xrange(len(reference)+1-n)])
        stats.append(max([sum((s_ngrams & r_ngrams).values()), 0]))
        stats.append(max([len(hypothesis)+1-n, 0]))
    return stats

In [24]:
def bleu(stats):
    if len(filter(lambda x: x==0, stats)) > 0:
        return 0
    (c, r) = stats[:2]
    log_bleu_prec = sum([math.log(float(x)/y) for x,y in zip(stats[2::2],stats[3::2])]) / 4.
    return math.exp(min([0, 1-float(r)/c]) + log_bleu_prec)

In [25]:
def get_validation_bleu(hypotheses, reference):
    stats = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
    #ref_lines = [line.strip().split() for line in open(path_to_dev_tgt, 'r')]
    for hyp, ref in zip(hypotheses, reference):
        stats += np.array(bleu_stats(hyp, ref))
    return "%.2f" % (100*bleu(stats))

In [26]:
def get_train_predictions():
    train_predictions = []    
    for ind, sent in enumerate(train_sentences[800:815]):
        
        if ind % 300 == 0:
            print ind, len(dev_sentences)
        sent = sent[0][0].split()
        src_words = np.array([source_word2ind[x] for x in sent]).astype(np.int32)
        stack = [source_word2ind['<ss>'], source_word2ind['</se>']]
        
        sentence_output = []
        for ind, word in enumerate(src_words):
            stack.insert(-1, word) 
            forward_context = src_words[ind:]
            forward_context_words = [source_word2ind['<fcs>']] + [x for x in forward_context] + [source_word2ind['</fcs>']]
            current_outputs = [target_word2ind['<s>']]
            action = f_get_action(np.array(stack).astype(np.int32), np.array(forward_context_words).astype(np.int32))
            
            if action > 0.5 or ind == len(src_words) -1: # If translate or last word of sentence
                while True:
                    next_word = f_eval_seq_seq(stack, current_outputs).argmax(axis=1)[-1]
                    current_outputs.append(next_word)
                    #print [target_ind2word[x] for x in current_outputs]
                    if next_word == target_word2ind['</s>'] or len(current_outputs) >= 15:
                        if next_word == target_word2ind['</s>']:
                            sentence_output.extend([target_ind2word[x] for x in current_outputs[1:-1]])
                        else:
                            sentence_output.extend([target_ind2word[x] for x in current_outputs[1:]])
                        break
                stack = [source_word2ind['<ss>'], source_word2ind['</se>']]
            else:
                pass
            print u'{: <2} | {: <20} | {: <2} | {: <30} | {: <30} '.format(str(ind), source_ind2word[word], 'T' if action > 0.5 else 'S', ' '.join([source_ind2word[x] for x in stack_copy]), ' '.join([target_ind2word[x] for x in current_outputs]))
        
        train_predictions.append(sentence_output)
        #print ' '.join(sentence_output)
        print '===================================================================================='
    return train_predictions

In [27]:
xx = ['<s>', 'the', '</s>']

In [28]:
xx.insert(-1, 'boy')

In [29]:
xx.insert(-1, 'runs')

In [30]:
xx

['<s>', 'the', 'boy', 'runs', '</s>']

In [31]:
def get_validation_predictions():
    validation_predictions = []
    NUM_MAX_SHIFT = 3
    correct_actions = 0
    incorrect_actions = 0
    correct_actions_t = 0
    incorrect_actions_t = 0
    for ind, sentence in enumerate(dev_sentences[800:815]):
        actionset = []
        if ind % 300 == 0:
            print ind, len(dev_sentences)
        sent = sentence[0][0].split()
        src_words = np.array([source_word2ind[x] for x in sent]).astype(np.int32)
        stack = [source_word2ind['<ss>'], source_word2ind['</se>']]
        assert len(src_words) == len(sentence[2:])
        sentence_output = []
        for ind, (word, gold) in enumerate(zip(src_words, sentence[2:])):
            stack.insert(-1, word) 
            forward_context = src_words[ind:]
            forward_context_words = [source_word2ind['<fcs>']] + [x for x in forward_context] + [source_word2ind['</fcs>']]
            current_outputs = [target_word2ind['<s>']]
            action = f_get_action(np.array(stack).astype(np.int32), np.array(forward_context_words).astype(np.int32))
            actionset.append('T' if action > 0.5 else 'S')
            if (gold[1] == 'T' and action > 0.5) or (gold[1] == 'S' and action < 0.5):
                correct_actions += 1
            else:
                incorrect_actions += 1
            if gold[1] == 'T':
                correct_actions_t += 1
            else:
                incorrect_actions_t += 1
            if action > 0.5 or ind == len(src_words) -1 or actionset[-NUM_MAX_SHIFT:] == ['S'] * NUM_MAX_SHIFT: # If translate or last word of sentence
                while True:
                    next_word = f_eval_seq_seq(stack, current_outputs).argmax(axis=1)[-1]
                    current_outputs.append(next_word)
                    #print [target_ind2word[x] for x in current_outputs]
                    if next_word == target_word2ind['</s>'] or len(current_outputs) >= 15:
                        if next_word == target_word2ind['</s>']:
                            sentence_output.extend([target_ind2word[x] for x in current_outputs[1:-1]])
                        else:
                            sentence_output.extend([target_ind2word[x] for x in current_outputs[1:]])
                        break
                stack_copy = copy.deepcopy(stack)
                stack = [source_word2ind['<ss>'], source_word2ind['</se>']]
            else:
                stack_copy = copy.deepcopy(stack)
            print u'{: <2} | {: <20} | {: <2} | {: <30} | {: <30} '.format(str(ind), source_ind2word[word], 'T' if action > 0.5 else 'S', ' '.join([source_ind2word[x] for x in stack_copy]), ' '.join([target_ind2word[x] for x in current_outputs]))
        
        validation_predictions.append(sentence_output)
        #print ' '.join(sentence_output)
        print '===================================================================================='
    model_acc = float(correct_actions) / float(correct_actions + incorrect_actions)
    baseline_acc = float(correct_actions_t) / float(correct_actions_t + incorrect_actions_t)
    print 'Action prediction accuracy : %f%% ' % (model_acc)
    print 'Action prediction accuracy Translate only : %f%% ' % (baseline_acc)
    return validation_predictions, model_acc, baseline_acc

In [58]:
def get_validation_predictions_noprint():
    validation_predictions = []
    NUM_MAX_SHIFT = 3
    correct_actions = 0
    incorrect_actions = 0
    correct_actions_t = 0
    incorrect_actions_t = 0
    all_actions = []
    for ind, sentence in enumerate(dev_sentences):
        actionset = []
        sentence_actions = []
        if ind % 300 == 0:
            print ind, len(dev_sentences)
        sent = sentence[0][0].split()
        src_words = np.array([source_word2ind[x] for x in sent]).astype(np.int32)
        stack = [source_word2ind['<ss>'], source_word2ind['</se>']]
        assert len(src_words) == len(sentence[2:])
        sentence_output = []
        for ind, (word, gold) in enumerate(zip(src_words, sentence[2:])):
            stack.insert(-1, word) 
            forward_context = src_words[ind:]
            forward_context_words = [source_word2ind['<fcs>']] + [x for x in forward_context] + [source_word2ind['</fcs>']]
            current_outputs = [target_word2ind['<s>']]
            action = f_get_action(np.array(stack).astype(np.int32), np.array(forward_context_words).astype(np.int32))
            actionset.append('T' if action > 0.5 else 'S')
            if (gold[1] == 'T' and action > 0.5) or (gold[1] == 'S' and action < 0.5):
                correct_actions += 1
            else:
                incorrect_actions += 1
            if gold[1] == 'T':
                correct_actions_t += 1
            else:
                incorrect_actions_t += 1
            if action > 0.5 or ind == len(src_words) -1 or actionset[-NUM_MAX_SHIFT:] == ['S'] * NUM_MAX_SHIFT: # If translate or last word of sentence
                while True:
                    next_word = f_eval_seq_seq(stack, current_outputs).argmax(axis=1)[-1]
                    current_outputs.append(next_word)
                    # print [target_ind2word[x] for x in current_outputs]
                    if next_word == target_word2ind['</s>'] or len(current_outputs) >= 15:
                        if next_word == target_word2ind['</s>']:
                            sentence_output.extend([target_ind2word[x] for x in current_outputs[1:-1]])
                        else:
                            sentence_output.extend([target_ind2word[x] for x in current_outputs[1:]])
                        break
                stack_copy = copy.deepcopy(stack)
                stack = [source_word2ind['<ss>'], source_word2ind['</se>']]
            else:
                stack_copy = copy.deepcopy(stack)
            # print u'{: <2} | {: <20} | {: <2} | {: <30} | {: <30} '.format(str(ind), source_ind2word[word], 'T' if action > 0.5 else 'S', ' '.join([source_ind2word[x] for x in stack_copy]), ' '.join([target_ind2word[x] for x in current_outputs]))
            sentence_actions.append([str(ind), source_ind2word[word], 'T' if action > 0.5 else 'S', ' '.join([source_ind2word[x] for x in stack_copy]), ' '.join([target_ind2word[x] for x in current_outputs])])
        validation_predictions.append(sentence_output)
        all_actions.append(sentence_actions)
        # print ' '.join(sentence_output)
        # print '===================================================================================='
    model_acc = float(correct_actions) / float(correct_actions + incorrect_actions)
    baseline_acc = float(correct_actions_t) / float(correct_actions_t + incorrect_actions_t)
    # print 'Action prediction accuracy : %f%% ' % (model_acc)
    # print 'Action prediction accuracy Translate only : %f%% ' % (baseline_acc)
    return validation_predictions, model_acc, baseline_acc, all_actions

In [33]:
for ind, sentence in enumerate(train_sentences[:1000]):
    trans = ''
    for state in sentence[2:]:
        translation = state[4].split()
        action = state[1]
        if action == 'T'and '$NONE$' in translation and translation != ['$NONE$']:
            print state[2], translation
        #if action == 'T' and translation == ['$NONE$']:
        #    print state[2], translation

In [34]:
best_valid_bleu = -sys.maxint

In [35]:
best_all_actions = []
best_action_acc = -sys.maxint
best_action_baseline = -sys.maxint

In [57]:
action_losses = []
seq_seq_losses = []
train_sentences = np.random.permutation(train_sentences)
for ind, sentence in enumerate(train_sentences):
    for state in sentence[2:]:
        word = state[0]
        action = state[1]
        stack_state = state[2].strip().split()
        forward_context = state[3].strip().split()
        translation = state[4].strip().split()
        stack_words = [source_word2ind['<ss>']] + [source_word2ind[word] for word in stack_state] + [source_word2ind['</se>']]
        forward_context_words = [source_word2ind['<fcs>']] + [source_word2ind[word] for word in forward_context] + [source_word2ind['</fcs>']]
        action = 1.0 if action == 'T' else 0.0
        action_loss, prediction = f_train_action(stack_words, forward_context_words, action)
        action_losses.append(action_loss)
        # Train seq-seq only if the model predicts 'T'
        if action == 1.0 and translation != ['$NONE$']:
            translation_words = [target_word2ind['<s>']] + [target_word2ind[word] for word in translation] + [target_word2ind['</s>']]
            seq_seq_loss = f_train_seq_seq(stack_words, translation_words[:-1], translation_words[1:])
            seq_seq_losses.append(seq_seq_loss)
        elif action == 0.0:
            pass
    if ind % 30 == 0:
        print 'Sentence %d out of %d action loss %f ' % (ind, len(sentences), np.mean(action_losses))
        print 'Sentence %d out of %d seq-seq loss %f ' % (ind, len(sentences), np.mean(seq_seq_losses))
    if ind % 500 == 0:
        valid_preds, action_acc, baseline_acc, all_actions = get_validation_predictions_noprint()
        valid_bleu = get_validation_bleu(valid_preds, [x[1][0].split() for x in dev_sentences[:300]])
        print 'Validation BLEU : %s ' % (valid_bleu)
        if valid_bleu > best_valid_bleu:
            best_valid_bleu = valid_bleu
            best_all_actions = all_actions
            best_action_acc = action_acc
            best_action_baseline = baseline_acc

Sentence 0 out of 155362 action loss 0.333333 
Sentence 0 out of 155362 seq-seq loss 3.515689 
0 3108
Validation BLEU : 3.73 
Sentence 30 out of 155362 action loss 0.369681 
Sentence 30 out of 155362 seq-seq loss 3.570181 
Sentence 60 out of 155362 action loss 0.415441 
Sentence 60 out of 155362 seq-seq loss 3.598959 
Sentence 90 out of 155362 action loss 0.425649 
Sentence 90 out of 155362 seq-seq loss 3.573635 
Sentence 120 out of 155362 action loss 0.412776 
Sentence 120 out of 155362 seq-seq loss 3.588873 
Sentence 150 out of 155362 action loss 0.401475 
Sentence 150 out of 155362 seq-seq loss 3.565289 
Sentence 180 out of 155362 action loss 0.393246 
Sentence 180 out of 155362 seq-seq loss 3.577946 
Sentence 210 out of 155362 action loss 0.401812 
Sentence 210 out of 155362 seq-seq loss 3.610161 
Sentence 240 out of 155362 action loss 0.406521 
Sentence 240 out of 155362 seq-seq loss 3.621521 
Sentence 270 out of 155362 action loss 0.402627 
Sentence 270 out of 155362 seq-seq loss

KeyboardInterrupt: 

In [51]:
best_valid_bleu

'5.44'

In [ ]:
valid_preds, action_acc, baseline_acc, all_actions = get_validation_predictions_noprint()

0 3108
300 3108
600 3108
900

In [195]:
valid_preds = get_validation_predictions_noprint()

0 3108


In [256]:
get_validation_bleu(valid_preds, [x[1][0].split() for x in dev_sentences[800:815]])

'0.00'

In [47]:
get_validation_predictions()

0 3108
0  | to                   | T  | <ss> to </se>                  | <s> à </s>                     
1  | be                   | T  | <ss> be </se>                  | <s> être </s>                  
2  | sure                 | T  | <ss> sure </se>                | <s> comment </s>               
3  | ,                    | T  | <ss> , </se>                   | <s> , </s>                     
4  | Russia               | T  | <ss> Russia </se>              | <s> la Russie </s>             
5  | has                  | T  | <ss> has </se>                 | <s> a </s>                     
6  | tightened            | T  | <ss> tightened </se>           | <s> la pression </s>           
7  | its                  | T  | <ss> its </se>                 | <s> son </s>                   
8  | control              | T  | <ss> control </se>             | <s> contrôle </s>              
9  | of                   | T  | <ss> of </se>                  | <s> de </s>                    
10 | the     

([[u'\xe0',
   u'\xeatre',
   u'comment',
   u',',
   u'la',
   u'Russie',
   u'a',
   u'la',
   u'pression',
   u'son',
   u'contr\xf4le',
   u'de',
   u'la',
   u'la',
   u'pression',
   u'la',
   u'distribution',
   u'de',
   u'la',
   u'la',
   u'pression',
   u'et',
   u'la',
   u'dictature',
   u'.',
   u'il',
   u'vers',
   u'la',
   u'la',
   u'pression',
   u'la',
   u'puissance',
   u'militaire',
   u',',
   u'la',
   u'pression',
   u'la',
   u'distribution',
   u'l',
   u'\u2019',
   u'emploi',
   u'la',
   u'afin',
   u'l',
   u'\u2019',
   u'\xe9conomie',
   u',',
   u'et',
   u'vers',
   u'afin',
   u'vers',
   u'dans',
   u'la',
   u'la',
   u'alliance',
   u'.'],
  [u'pour',
   u'l',
   u'\u2019',
   u'ann\xe9es',
   u',',
   u'il',
   u'a',
   u'vers',
   u'tous',
   u'les',
   u'la',
   u'distribution',
   u'la',
   u'pression',
   u'\xe0',
   u'suivre',
   u'la',
   u'pression',
   u'vers',
   u',',
   u'la',
   u'pression',
   u'un',
   u'essentiel',
   u'la',
   u

In [53]:
len(sentences)

155362

In [55]:
len(test_sentences)

4661

In [56]:
len(train_sentences)

147593